In [1]:
import os
import json
from pathlib import Path

while ".git" not in os.listdir(os.getcwd()): prj_dir = os.chdir(Path(os.getcwd())/"..")

from owslib.wmts import WebMapTileService

In [2]:
# URL of the WMTS service
url = 'https://mapsneu.wien.gv.at/basemapneu/1.0.0/WMTSCapabilities.xml'

# Connect to the WMTS service
wmts = WebMapTileService(url)

print(vars(wmts.provider))
print(wmts.contents)

{'_root': <Element {http://www.opengis.net/ows/1.1}ServiceProvider at 0x7fd8ac2dd740>, 'name': 'City of Vienna', 'contact': <owslib.ows.ServiceContact object at 0x7fd8ac23d890>, 'url': 'https://www.wien.gv.at/viennagis'}
{'geolandbasemap': <owslib.wmts.ContentMetadata object at 0x7fd896743150>, 'bmapoverlay': <owslib.wmts.ContentMetadata object at 0x7fd896741390>, 'bmapgrau': <owslib.wmts.ContentMetadata object at 0x7fd896741a90>, 'bmaphidpi': <owslib.wmts.ContentMetadata object at 0x7fd8b03fb410>, 'bmaporthofoto30cm': <owslib.wmts.ContentMetadata object at 0x7fd8967437d0>, 'bmapgelaende': <owslib.wmts.ContentMetadata object at 0x7fd896748bd0>, 'bmapoberflaeche': <owslib.wmts.ContentMetadata object at 0x7fd896748e90>}


In [3]:
for n, l in wmts.contents.items():
    print(f'- {n}')
    print(vars(l))

- geolandbasemap
{'parent': None, 'index': '1', 'id': 'geolandbasemap', 'name': 'geolandbasemap', 'title': 'Geoland Basemap', 'abstract': 'Basemap von Österreich in Farbe', 'boundingBox': [], 'boundingBoxWGS84': (8.782379, 46.35877, 17.5, 49.037872), '_tilematrixsets': ['google3857'], 'tilematrixsetlinks': {'google3857': <TileMatrixSetLink: google3857, tilematrixlimits={...}>}, 'resourceURLs': [{'format': 'image/png', 'resourceType': 'tile', 'template': 'https://mapsneu.wien.gv.at/basemap/geolandbasemap/{Style}/{TileMatrixSet}/{TileMatrix}/{TileRow}/{TileCol}.png'}], 'styles': {'normal': {'isDefault': True}}, 'formats': ['image/png'], 'keywords': [], 'infoformats': [], 'layers': []}
- bmapoverlay
{'parent': None, 'index': '2', 'id': 'bmapoverlay', 'name': 'bmapoverlay', 'title': 'Geoland Basemap Overlay', 'abstract': 'Basemap von Österreich nur Beschriftung als transparenter Layer', 'boundingBox': [], 'boundingBoxWGS84': (8.782379, 46.35877, 17.5, 49.037872), '_tilematrixsets': ['googl

In [5]:
layers = {k: {
    'name': l.name,
    'style': list(l.styles.keys())[0],
    'url': l.resourceURLs[0]['template']
} for k, l in wmts.contents.items()}

for l in layers.values():
    replace = {
        '{TileRow}': '{y}',
        '{TileCol}': '{x}',
        '{TileMatrix}': '{z}',
        '{Style}': l['style'],
        '{TileMatrixSet}': 'google3857',
    }
    
    for s, r in replace.items():
        l['url'] = l['url'].replace(s, r)

layers

{'geolandbasemap': {'name': 'geolandbasemap',
  'style': 'normal',
  'url': 'https://mapsneu.wien.gv.at/basemap/geolandbasemap/normal/google3857/{z}/{y}/{x}.png'},
 'bmapoverlay': {'name': 'bmapoverlay',
  'style': 'normal',
  'url': 'https://mapsneu.wien.gv.at/basemap/bmapoverlay/normal/google3857/{z}/{y}/{x}.png'},
 'bmapgrau': {'name': 'bmapgrau',
  'style': 'normal',
  'url': 'https://mapsneu.wien.gv.at/basemap/bmapgrau/normal/google3857/{z}/{y}/{x}.png'},
 'bmaphidpi': {'name': 'bmaphidpi',
  'style': 'normal',
  'url': 'https://mapsneu.wien.gv.at/basemap/bmaphidpi/normal/google3857/{z}/{y}/{x}.jpeg'},
 'bmaporthofoto30cm': {'name': 'bmaporthofoto30cm',
  'style': 'normal',
  'url': 'https://mapsneu.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg'},
 'bmapgelaende': {'name': 'bmapgelaende',
  'style': 'grau',
  'url': 'https://mapsneu.wien.gv.at/basemap/bmapgelaende/grau/google3857/{z}/{y}/{x}.jpeg'},
 'bmapoberflaeche': {'name': 'bmapoberflaeche',
  'style

In [11]:
with open('web/assets/data/ortho-cap-basemap.json', 'w') as f:
    json.dump(layers, f)